# Notebook 4 for Capstone assignment

*This notebook will be used for the Coursera Capstone assignment*

## Exploring Toronto
### Segmenting and Clustering Neighborhoods in Toronto assignment

*Duplicated from Notebook 3*

Install Beautiful Soup 4 to scrape wiki page and link to web page

In [1]:
# import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib import request

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

content = urlopen(url).read()
soup = BeautifulSoup(content)

print ("Document loaded into Beautiful Soup: ", soup.title.string)


Document loaded into Beautiful Soup:  List of postal codes of Canada: M - Wikipedia


Upload wiki table into a csv file, dressing 'Not assigned' data

In [2]:
req = urlopen(url)
article = req.read().decode()

with open('ISO_3166-1_alpha-2.html', 'w') as fo:
    fo.write(article)

# Load article, turn into soup and get the table.
article = open('ISO_3166-1_alpha-2.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:5] == ['Postcode', 'Borough', 'Neighbourhood', 'ISO 3166-2']:
        break

# Extract the columns we want and write to a semicolon-delimited text file.
with open('iso_3166-1_alpha-2_codes.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        Postcode, Borough, Neighbourhood = [td.text.strip() for td in tds[:3]]

        # extract rows where borough unassigned and use borogh name where Neighbourhood unassigned
        if 'Not assigned' in Borough:
            continue 
        if 'Not assigned' in Neighbourhood:
            Neighbourhood = Borough
        print('; '.join([Postcode, Borough, Neighbourhood]), file=fo)

Read csv file into dataframe

In [3]:
df = pd.read_csv('iso_3166-1_alpha-2_codes.txt', sep=';', header=None, names=['Postcode', 'Borough', 'Neighbourhood'])

Combine Neighbourhoods in same Postcode and drop resulting duplicate postcodes

In [4]:
for i,row in df.iterrows():
    if i==0: 
        continue
    if (df.at[i,'Postcode'] == df.at[i-1,'Postcode']):
        df.at[i,'Neighbourhood'] = df.at[i-1,'Neighbourhood'] + "," + df.at[i,'Neighbourhood']
        df.at[i-1,'Postcode'] = 'Delete'
                                     
df.drop(df[df.Postcode == 'Delete'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
print("Head of cleaned Dataframe: ")
df.head()

Head of cleaned Dataframe: 


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [5]:
print("Check the size/shape of the dataframe:")
df.shape

Check the size/shape of the dataframe:


(103, 3)

## Adding Geospacial data

Upload csv file (geocoder would not install on Watson)

In [6]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df = geo_df.rename(columns = {"Postal Code":"Postcode"})
geo_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Add geospacial data to Canada dataframe

In [7]:
Canada_df = pd.merge(left=df, right=geo_df)
Canada_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [8]:
Canada_df.shape

(103, 5)

## Exploring and clustering the neighborhoods in Toronto

Examine the size of the dataset we have on Toronto

In [9]:
print('The Canada dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Canada_df['Borough'].unique()),
        Canada_df.shape[0]
    )
)

The Canada dataframe has 11 boroughs and 103 neighborhoods.


Create a Toronto subset of the Canada dataframe based on Borough name 

In [15]:
Toronto_df=Canada_df[Canada_df['Borough'].str.contains("Toronto")]
Toronto_df.reset_index(drop=True, inplace=True)
Toronto_df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Sta...",43.640816,-79.381752


In [16]:
print('The Toronto dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_df['Borough'].unique()),
        Toronto_df.shape[0]
    )
)

The Toronto dataframe has 4 boroughs and 38 neighborhoods.


Download the various libraries required

In [17]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Use geopy library to get the latitude and longitude values of New York City

In [18]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create map of Toronto using latitude and longitude values

In [21]:
# define Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto